## Deliverable 2. Create a Customer Travel Destinations Map.

In [43]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [44]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Rikitea,PF,-23.12,-134.97,70.02,71,40,8.12,Clouds
1,1,Paragominas,BR,-2.97,-47.48,70.48,98,51,2.64,Clouds
2,2,Bambous Virieux,MU,-20.34,57.76,73.99,78,75,8.05,Clouds
3,3,Deniliquin,AU,-35.53,144.97,60.01,50,0,1.01,Clear
4,4,Chokurdakh,RU,70.63,147.92,44.64,75,100,7.25,Clouds


In [45]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [46]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Rikitea,PF,-23.12,-134.97,70.02,71,40,8.12,Clouds
1,1,Paragominas,BR,-2.97,-47.48,70.48,98,51,2.64,Clouds
2,2,Bambous Virieux,MU,-20.34,57.76,73.99,78,75,8.05,Clouds
5,5,Tautira,PF,-17.73,-149.15,77.00,73,20,3.36,Clouds
11,11,Auki,SB,-8.77,160.70,80.55,79,100,6.04,Clouds
12,12,Pangai,TO,-19.80,-174.35,78.80,74,20,13.87,Clouds
13,13,Atuona,PF,-9.80,-139.03,79.03,74,60,12.12,Rain
17,17,Bitung,ID,1.45,125.18,91.40,55,20,11.41,Clouds
18,18,Khatanga,RU,71.97,102.50,72.12,39,24,7.56,Clouds
19,19,Fort-Shevchenko,KZ,44.51,50.26,77.72,40,0,16.62,Clear


In [47]:
# 4a. Determine if there are any empty rows.
#4.5.1.
preferred_cities_df.isnull()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False
11,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
682,False,False,False,False,False,False,False,False,False,False
683,False,False,False,False,False,False,False,False,False,False
686,False,False,False,False,False,False,False,False,False,False
688,False,False,False,False,False,False,False,False,False,False


In [48]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [49]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Weather", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
0,Rikitea,PF,70.02,Clouds,-23.12,-134.97,
1,Paragominas,BR,70.48,Clouds,-2.97,-47.48,
2,Bambous Virieux,MU,73.99,Clouds,-20.34,57.76,
5,Tautira,PF,77.00,Clouds,-17.73,-149.15,
11,Auki,SB,80.55,Clouds,-8.77,160.70,
12,Pangai,TO,78.80,Clouds,-19.80,-174.35,
13,Atuona,PF,79.03,Rain,-9.80,-139.03,
17,Bitung,ID,91.40,Clouds,1.45,125.18,
18,Khatanga,RU,72.12,Clouds,71.97,102.50,
19,Fort-Shevchenko,KZ,77.72,Clear,44.51,50.26,


In [50]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")
hotel_df.head(10)

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
0,Rikitea,PF,70.02,Clouds,-23.12,-134.97,Pension Maro'i
1,Paragominas,BR,70.48,Clouds,-2.97,-47.48,HOTEL RODA VIVA
2,Bambous Virieux,MU,73.99,Clouds,-20.34,57.76,Casa Tia Villa
5,Tautira,PF,77.00,Clouds,-17.73,-149.15,Pension Oaoa
11,Auki,SB,80.55,Clouds,-8.77,160.70,Auki Motel
12,Pangai,TO,78.80,Clouds,-19.80,-174.35,Ha'apai Beach Resort
13,Atuona,PF,79.03,Rain,-9.80,-139.03,Villa Enata
17,Bitung,ID,91.40,Clouds,1.45,125.18,Peoplesight Learning Center ( PLC )
18,Khatanga,RU,72.12,Clouds,71.97,102.50,"Gostinitsa ""Zapolyar'ye"""
19,Fort-Shevchenko,KZ,77.72,Clear,44.51,50.26,Oasis hotel Bautino


In [53]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
0,Rikitea,PF,70.02,Clouds,-23.12,-134.97,Pension Maro'i
1,Paragominas,BR,70.48,Clouds,-2.97,-47.48,HOTEL RODA VIVA
2,Bambous Virieux,MU,73.99,Clouds,-20.34,57.76,Casa Tia Villa
5,Tautira,PF,77.00,Clouds,-17.73,-149.15,Pension Oaoa
11,Auki,SB,80.55,Clouds,-8.77,160.70,Auki Motel
...,...,...,...,...,...,...,...
682,Cantapoy,PH,83.73,Clouds,9.49,125.44,Brgy Pili Mountain View
683,Pontes E Lacerda,BR,75.45,Clouds,-15.23,-59.34,Hotel Bressan
686,Kaberamaido,UG,70.23,Clouds,1.74,33.16,"Little Orchard, Kaberamaido"
688,Sorrento,IT,78.01,Clear,40.63,14.37,Hilton Sorrento Palace


In [60]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [58]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [59]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))